<a href="https://colab.research.google.com/github/Amplil/laser/blob/master/laser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [LASER for NLP tasks - Part 2](https://www.engati.com/blog/laser-for-nlp-tasks-part-ii)のページを参考にしたプログラム

## インストール（1度goole driveに保存したら2回目以降は実行する必要ない。）

In [ ]:
#!git clone https://github.com/facebookresearch/LASER

In [ ]:
#cd "/content/drive/My Drive/Colab Notebooks/LASER_official"

In [ ]:
#ls

In [ ]:
#!export LASER="/content/drive/My Drive/Colab Notebooks/LASER"

In [ ]:
#!export LASER="${HOME}/projects/laser"

In [ ]:
#!bash ./install_models.sh

In [ ]:
# !pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" laserembeddings

In [ ]:
# cd "/content/drive/MyDrive/Colab Notebooks/packages"

In [ ]:
#!pip install laserembeddings

In [ ]:
#!python -m laserembeddings download-models

In [ ]:
#!python -m laserembeddings download-test-data

In [ ]:
#!pytest tests/test_laser.py

In [ ]:
#!pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" laserembeddings[ja]

In [ ]:
#!pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" laserembeddings[zh]

In [ ]:
#!pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" boto3

In [2]:
#!pip install laserembeddings

     |████████████████████████████████| 860kB 12.8MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp37-none-any.whl size=883999 sha256=9c32b32470140ed78d0551c55cd9c5b88dbc727ef2fa302ce17b2bef23c6c69a
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


## インポート

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
import re
import pandas as pd
from laserembeddings import Laser

## kaggleのコンペ[Tweet Sentiment Extractionのtest.csv](https://www.kaggle.com/c/tweet-sentiment-extraction/data?select=test.csv)を使う

In [ ]:
#df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/tweet_train.csv', index_col=0).fillna('')
#df_train

In [1]:
%ls

 �h���C�u C �̃{�����[�� ���x��������܂���B
 �{�����[�� �V���A���ԍ��� 0C1D-FA87 �ł�

 c:\Users\hara\Documents\programming_local\laser �̃f�B���N�g��

2021/04/08  18:43    <DIR>          .
2021/04/08  18:43    <DIR>          ..
2021/04/08  18:43            57,227 laser.ipynb
2021/03/23  22:23           987,712 tweet_train.csv
               2 �̃t�@�C��           1,044,939 �o�C�g
               2 �̃f�B���N�g��  899,247,636,480 �o�C�g�̋󂫗̈�


In [5]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [6]:
df_train = pd.read_csv('train.csv', index_col=0).fillna('')
df_train

,text,selected_text,sentiment
textID,,,
cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
088c60f138,my boss is bullying me...,bullying me,negative
9642c003ef,what interview! leave me alone,leave me alone,negative
358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...
4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive


In [7]:
df_train.shape

(27481, 3)

In [8]:
!pip install icecream

In [9]:
from icecream import ic
datax = df_train["selected_text"]
datay = df_train["sentiment"]
ic(datax.shape)
ic(datay.shape)

ic| datax.shape: (27481,)
ic| datay.shape: (27481,)


(27481,)

In [10]:
from sklearn.model_selection import train_test_split
trainx, testx, trainy, testy = train_test_split(datax, datay, test_size=0.3)
ic(trainx.shape)
ic(testx.shape)


ic| trainx.shape: (19236,)
ic| testx.shape: (8245,)


(8245,)

In [11]:
!pip install pyforest

  Created wheel for pyforest: filename=pyforest-1.0.3-py2.py3-none-any.whl size=13716 sha256=0ed7d8b4ef716e729f11098f86ce0cc4d0af74ba0b37da2aa9b949d8997ba1e5
  Stored in directory: /root/.cache/pip/wheels/af/0b/39/340a7f15fc8d4ff5ab50847b28789afea04452a9d51b9721b5
Successfully built pyforest


In [13]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


# =======Start Line=========

In [14]:
from laserembeddings import Laser
import pyforest
trainx_tokenization = np.array(Laser().embed_sentences(trainx, lang='en'))
testx_tokenization = np.array(Laser().embed_sentences(testx, lang='en'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
ic(trainx_tokenization.shape)
ic(testx_tokenization.shape)

ic| trainx_tokenization.shape: (19236, 1024)
ic| testx_tokenization.shape: (8245, 1024)


(8245, 1024)

## 例

In [17]:
model_test = sklearn.linear_model.LogisticRegression()
model_test.fit(trainx_tokenization, trainy)
trainy_pred = model_test.predict(trainx_tokenization)
testy_pred = model_test.predict(testx_tokenization)
ic(testy_pred[:5])
print("accuracy on training data:",sklearn.metrics.accuracy_score(trainy, trainy_pred))
print("accuracy on testing data:",sklearn.metrics.accuracy_score(testy, testy_pred))
print("classification report on testing data:")
print(sklearn.metrics.classification_report(testy,testy_pred))

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
ic| testy_pred[:5]: array(['positive', 'neutral', 'neutral', 'negative', 'neutral'],
                          dtype=object)


<IPython.core.display.Javascript object>

accuracy on training data: 0.8619255562487004


<IPython.core.display.Javascript object>

accuracy on testing data: 0.852274105518496
classification report on testing data:


<IPython.core.display.Javascript object>

              precision    recall  f1-score   support

    negative       0.85      0.80      0.82      2326
     neutral       0.83      0.90      0.86      3341
    positive       0.90      0.84      0.87      2578

    accuracy                           0.85      8245
   macro avg       0.86      0.85      0.85      8245
weighted avg       0.85      0.85      0.85      8245



## Keras deeplearning

In [18]:
# trainy_ = np.array(trainy)
# ic(trainy_.shape)

In [19]:
ic(type(trainy))
# trainy.loc[trainy['sentiment']== 'positive', 'sentiment']=2
values = np.array(trainy)
print(values)
label_encoder = sklearn.preprocessing.LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
onehot_encoder = sklearn.preprocessing.OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
# inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
# print(inverted)

ic| type(trainy): <class 'pandas.core.series.Series'>


<IPython.core.display.Javascript object>

['neutral' 'neutral' 'positive' ... 'neutral' 'negative' 'negative']


<IPython.core.display.Javascript object>

[1 1 2 ... 1 0 0]


<IPython.core.display.Javascript object>

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [23]:
def oh_ecoder(x):
  values = np.array(x)
  # print(values)
  label_encoder = sklearn.preprocessing.LabelEncoder()
  integer_encoded = label_encoder.fit_transform(values)
  # print(integer_encoded)
  onehot_encoder = sklearn.preprocessing.OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
  # print(onehot_encoded)
  return onehot_encoded

In [27]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape= (1024,)))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(3, activation="softmax"))
model.compile(loss= tf.keras.losses.categorical_crossentropy,
       optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),
       metrics = ["accuracy"]
    )
model.fit(trainx_tokenization, ic(oh_ecoder(trainy)), batch_size= 32, epochs= 50, callbacks=[])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ic| oh_ecoder(trainy): array([[0., 1., 0.],
                              [0., 1., 0.],
                              [0., 0., 1.],
                              ...,
                              [0., 1., 0.],
                              [1., 0., 0.],
                              [1., 0., 0.]])


Epoch 1/50
602/602 [==============================] - 2s 3ms/step - loss: 1.0172 - accuracy: 0.5390
Epoch 2/50
602/602 [==============================] - 2s 3ms/step - loss: 0.7292 - accuracy: 0.6675
Epoch 3/50
602/602 [==============================] - 2s 3ms/step - loss: 0.5526 - accuracy: 0.8076
Epoch 4/50
602/602 [==============================] - 2s 3ms/step - loss: 0.4211 - accuracy: 0.8500
Epoch 5/50
602/602 [==============================] - 2s 3ms/step - loss: 0.4080 - accuracy: 0.8523
Epoch 6/50
602/602 [==============================] - 2s 3ms/step - loss: 0.3814 - accuracy: 0.8624
Epoch 7/50
602/602 [==============================] - 2s 3ms/step - loss: 0.3762 - accuracy: 0.8616
Epoch 8/50
602/602 [==============================] - 2s 3ms/step - loss: 0.3734 - accuracy: 0.8649
Epoch 9/50
602/602 [==============================] - 2s 3ms/step - loss: 0.3683 - accuracy: 0.8677
Epoch 10/50
602/602 [==============================] - 2s 3ms/step - loss: 0.3543 - accuracy: 0.8721

In [28]:
print("accuracy on training data:", model.evaluate(trainx_tokenization, oh_ecoder(trainy)))
print("accuracy on validation data:",model.evaluate(testx_tokenization, oh_ecoder (testy)))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

602/602 [==============================] - 1s 2ms/step - loss: 0.0601 - accuracy: 0.9880
accuracy on training data: [0.06009500101208687, 0.9879912734031677]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

258/258 [==============================] - 1s 2ms/step - loss: 0.6662 - accuracy: 0.8479
accuracy on validation data: [0.6661946177482605, 0.8479078412055969]


In [29]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_25 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_26 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_27 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_28 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_29 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_30 (Dense)             (None, 128)              

# =======End Line=========

In [ ]:
#特徴データ
feature = df_train.text
#ラベルデータ
target = df_train.sentiment

In [ ]:
#scikit-learnを用いたデータ分割
from sklearn.model_selection import train_test_split
X1, X2, Y1, Y2 = train_test_split(feature, target, test_size=0.3)

In [ ]:
X1

In [ ]:
Y1

In [ ]:
from laserembeddings import Laser
laser=Laser()
embeddings = laser.embed_sentences(
    ['let your neural network be polyglot',
     'use multilingual embeddings!'],
    lang='en')  # lang is only used for tokenization
embeddings

In [ ]:
laser=Laser()
embeddings = laser.embed_sentences(
    ['let your neural network be polyglot'],
    lang='en')  # lang is only used for tokenization
embeddings

In [ ]:
model=Sequential()
model.add(Dense(3, input_dim = len(X1.index), activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(ic(X1),Y1,validation_data=(X2,Y2),epochs=5)

In [ ]:
model.summary()

In [ ]:
model.fit(X1,Y1,validation_data=(X2,Y2),epochs=5)

In [ ]:
model.save('laser_model.h5')

In [ ]:
model=Sequential()
model.add(Dense(3,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X1,Y1,validation_data=(X2,Y2),epochs=5)

In [ ]:
embeddings = laser.embed_sentences(input_dict,
    lang='en')  # lang is only used for tokenization
embeddings

In [ ]:
embedded_dict = {}
for key in input_dict:
    embedded_dict[key] = np.array(get_vect(input_dict[key], lang = key))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_lib = cosine_similarity(embedded_dict['it'], embedded_dict['es'])

In [ ]:
embeddings[0].shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_lib = cosine_similarity(embeddings[0], embeddings[1])

In [ ]:
embeddings[0]

# [Multilingual Sentence Models in NLP](https://towardsdatascience.com/multilingual-sentence-models-in-nlp-476f1f246d2f)のページを参考にしたプログラム

In [ ]:
import requests
import numpy as np
def get_vect(query_in, lang = 'en', address = '127.0.0.1:8050'):
    url = "http://" + address + "/vectorize"
    params = {"q": query_in, "lang": lang}
    resp = requests.get(url=url, params=params).json()
    return resp["embedding"]

In [ ]:
input_dict = {
    "en":"Machine learning isn't as hard as people think.",
    "de":"Maschinelles Lernen ist nicht so schwer wie die Leute denken.",
    "fr":"L'apprentissage automatique n'est pas aussi difficile qu'on le pense.",
    "it":"L'apprendimento automatico non è così difficile come la gente pensa.",
    "es":"El aprendizaje automático no es tan difícil como la gente piensa.",
    "ru":"Машинное обучение не так сложно, как думают люди.",
    "ar":"التعلم الآلي ليس صعبا كما يظن الناس.",
    "pt":"O aprendizado de máquina não é tão difícil quanto as pessoas pensam.",
}

In [ ]:
embedded_dict = {}
for key in input_dict:
    embedded_dict[key] = np.array(get_vect(input_dict[key], lang = key))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_lib = cosine_similarity(embedded_dict['it'], embedded_dict['es'])

# [laserembeddings · PyPI](https://pypi.org/project/laserembeddings/)のページを参考にしたプログラム

In [ ]:
from laserembeddings import Laser

laser = Laser()

# if all sentences are in the same language:

embeddings = laser.embed_sentences(
    ['let your neural network be polyglot',
     'use multilingual embeddings!'],
    lang='en')  # lang is only used for tokenization

# embeddings is a N*1024 (N = number of sentences) NumPy array

In [ ]:
embeddings = laser.embed_sentences(
    ['I love pasta.',
     "J'adore les pâtes.",
     'Ich liebe Pasta.'],
    lang=['en', 'fr', 'de'])

In [ ]:
%%bash
# DOCS: https://github.com/facebookresearch/LASER/blob/master/install_models.sh

mkdir -p models/laser/
# for FILE in bilstm.eparl21.2018-11-19.pt eparl21.fcodes eparl21.fvocab bilstm.93langs.2018-12-26.pt 93langs.fcodes 93langs.fvocab; do
for FILE in bilstm.93langs.2018-12-26.pt 93langs.fcodes 93langs.fvocab; do
    wget -cq https://dl.fbaipublicfiles.com/laser/models/$FILE -O models/laser/$FILE
done

In [ ]:
from laserembeddings import Laser

path_to_bpe_codes = "./models/laser/93langs.fcodes"
path_to_bpe_vocab = "./models/laser/93langs.fvocab"
path_to_encoder ="./models/laser/bilstm.93langs.2018-12-26.pt"

laser = Laser(path_to_bpe_codes, path_to_bpe_vocab, path_to_encoder)

# you can also supply file objects instead of file paths

In [ ]:
from io import BytesIO, StringIO
from laserembeddings import Laser
import boto3

s3 = boto3.resource('s3')
MODELS_BUCKET = ...

f_bpe_codes = StringIO(s3.Object(MODELS_BUCKET, path_to_bpe_codes).get()['Body'].read().decode('utf-8'))
f_bpe_vocab = StringIO(s3.Object(MODELS_BUCKET, path_to_bpe_vocab).get()['Body'].read().decode('utf-8'))
f_encoder = BytesIO(s3.Object(MODELS_BUCKET, path_to_encoder).get()['Body'].read())

laser = Laser(f_bpe_codes, f_bpe_vocab, f_encoder)